In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexEntropy"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = ResNet50(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 2048)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 2048)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  2.6min


[CV] ........................... C=0.01, score=0.937992, total= 2.8min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.933267, total= 2.8min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.946429, total= 2.9min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.927734, total= 3.0min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.920588, total= 3.1min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.934442, total= 3.1min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.940535, total= 3.2min
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  4.7min


[CV] ............................ C=0.1, score=0.933333, total= 4.7min
[CV] ............................ C=0.1, score=0.927794, total= 4.7min
[CV] C=10.0 ..........................................................
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.930528, total= 4.7min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.933464, total= 4.7min
[CV] ............................ C=0.1, score=0.942517, total= 4.7min
[CV] ............................ C=0.1, score=0.947421, total= 4.7min
[CV] C=100.0 .........................................................
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.938247, total= 4.7min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.934570, total= 4.8min
[CV] .

...

[CV] ................... C=1000000000.0, score=0.927451, total= 4.9min
[CV] ................... C=1000000000.0, score=0.938553, total= 4.9min
[CV] ................... C=1000000000.0, score=0.927734, total= 5.1min
[CV] ................... C=1000000000.0, score=0.925816, total= 5.0min
[CV] ................... C=1000000000.0, score=0.935039, total= 5.0min
[CV] ................... C=1000000000.0, score=0.911417, total= 5.0min
[CV] ................... C=1000000000.0, score=0.944444, total= 5.0min
[CV] ................... C=1000000000.0, score=0.923459, total= 5.0min
[CV] ................... C=1000000000.0, score=0.934263, total= 4.9min
[CV] .................. C=10000000000.0, score=0.927593, total= 4.0min
[CV] .................. C=10000000000.0, score=0.926758, total= 4.4min
[CV] .................. C=10000000000.0, score=0.929412, total= 4.2min
[CV] .................. C=10000000000.0, score=0.912402, total= 4.2min
[CV] .................. C=10000000000.0, score=0.937562, total= 4.0min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 27.0min remaining:  1.1min


[CV] .................. C=10000000000.0, score=0.936024, total= 4.2min
[CV] .................. C=10000000000.0, score=0.926805, total= 4.2min
[CV] .................. C=10000000000.0, score=0.940476, total= 4.0min
[CV] .................. C=10000000000.0, score=0.925447, total= 3.9min
[CV] .................. C=10000000000.0, score=0.933267, total= 3.8min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 27.3min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([176.53772526, 279.21849   , 289.94694939, 298.65239005,
        302.08529527, 298.69810705, 300.03327456, 299.15640819,
        300.79453897, 300.15560939, 300.28456745, 298.17677495,
        244.20388069]),
 'mean_score_time': array([0.07514775, 0.06675634, 0.07114239, 0.05950301, 0.0593024 ,
        0.06039891, 0.06772227, 0.05788536, 0.06121356, 0.05715067,
        0.06384337, 0.0495266 , 0.04166245]),
 'mean_test_score': array([0.93182715, 0.93537885, 0.93034728, 0.93074191, 0.92916338,
        0.93074191, 0.92876875, 0.92985399, 0.92965667, 0.9300513 ,
        0.9300513 , 0.92955801, 0.92955801]),
 'mean_train_score': array([0.97716607, 0.99837766, 0.99976983, 0.99975883, 0.99967112,
        0.99973695, 0.9997479 , 0.99974791, 0.99973693, 0.99972598,
        0.99972596, 0.99973693, 0.99974788]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.93183, std: 0.00789, params: {'C': 0.01},
 mean: 0.93538, std: 0.00736, params: {'C': 0.1},
 mean: 0.93035, std: 0.00836, params: {'C': 1.0},
 mean: 0.93074, std: 0.00807, params: {'C': 10.0},
 mean: 0.92916, std: 0.00873, params: {'C': 100.0},
 mean: 0.93074, std: 0.00778, params: {'C': 1000.0},
 mean: 0.92877, std: 0.00852, params: {'C': 10000.0},
 mean: 0.92985, std: 0.00801, params: {'C': 100000.0},
 mean: 0.92966, std: 0.00791, params: {'C': 1000000.0},
 mean: 0.93005, std: 0.00796, params: {'C': 10000000.0},
 mean: 0.93005, std: 0.00791, params: {'C': 100000000.0},
 mean: 0.92956, std: 0.00864, params: {'C': 1000000000.0},
 mean: 0.92956, std: 0.00755, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.1} - score: 0.9354
